<a href="https://colab.research.google.com/github/txvuong150764/Sentence_Semantic_phoBERT/blob/master/QS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from tensorflow import keras
from keras.layers import Dense
import tensorflow as tf
import torch.nn as nn
import numpy as np
from tqdm import tqdm

In [225]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git
!pip install vncorenlp
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP('VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators='wseg', max_heap_size='-Xmx500m')

def word_segment(text):
    word_segmented_text = annotator.tokenize(text) 
    sentences = []
    for sentence in word_segmented_text:
        sentences.append(' '.join(sentence))
    return ' '.join(sentences)

fatal: destination path 'VnCoreNLP' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [226]:
s = word_segment("Ngày quốc khánh của việt nam là 2/9")
s

'Ngày quốc_khánh của việt_nam là 2/9'

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 6
max_length = 128

In [ ]:
import pandas as pd

dataset_df = pd.read_csv('/content/train.csv')
valid_df = pd.read_csv('/content/dev.csv')

In [ ]:
for i in range(len(dataset_df)):
    dataset_df['question 1'][i] = word_segment(dataset_df['question 1'][i])
    dataset_df['question 2'][i] = word_segment(dataset_df['question 2'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataset_df, test_size=0.3, random_state=42)

In [ ]:
test_df

,question 1,question 2,similarity
53261,lý thái tổ mất vào ngày_tháng năm nào,isaac newton mất vào ngày nào,False
123871,vì sao phải mặc quần_lót,vì sao lại mà phải mặc quần_lót,True
28417,ngày truyền_thống của quân_lực việt_nam cộng_h...,ngày truyền_thống của quân_lực việt_nam cộng_h...,False
33182,phim titanic phát_hành năm nào,bộ manga nào có lượt phát_hành kỷ_lục năm 2008,False
83411,bảy kỳ_quan thế_giới mới được lựa_chọn như_thế...,điều_kiện lựa_chọn thẩm_phán của mỗi_một viện,False
...,...,...,...
59060,cha_mẹ của luiz inácio lula da silva là ai,luiz inácio lula da silva giành thắng_lợi tron...,False
83077,bóng_chày được chơi phổ_biến ở đâu,bóng_rổ phổ_biến ở các nước nào,False
15603,korkishko bắt_đầu sự_nghiệp bóng_đá tại câu_lạ...,xavi từ_giã sự_nghiệp bóng_đá quốc_tế vào năm nào,False
19151,ai đứng đầu tây_sơn thất hổ_tướng,hổ_tướng nào nhà tây_sơn có vợ là nữ_tướng trầ...,False


In [ ]:
for i in range(len(valid_df)):
    valid_df['question 1'][i] = word_segment(valid_df['question 1'][i])
    valid_df['question 2'][i] = word_segment(valid_df['question 2'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(valid_df['question 1'][0], valid_df['question 2'][0])

văn_hoá việt_nam có từ bao_giờ tên gọi hai chữ việt_nam và quốc_hiệu việt_nam có từ bao_giờ


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class EquavilenceDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        """
        To customize dataset, inherit from Dataset class and implement
        __len__ & __getitem__
        __getitem__ should return 
            data:
                input_ids
                attention_masks
                text
                targets
        """
        row = self.df.iloc[index]
        text1, text2, label = self.get_input_data(row)

        # Encode_plus will:
        # (1) split text into token
        # (2) Add the '[CLS]' and '[SEP]' token to the start and end
        # (3) Truncate/Pad sentence to max length
        # (4) Map token to their IDS
        # (5) Create attention mask
        # (6) Return a dictionary of outputs
        encoding = self.tokenizer(
            text1,
            text2,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )
        
        return {
            'text1': text1,
            'text2': text2,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_masks': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long),
        }


    def labelencoder(self,text):
        if text==True:
            return 1
        elif text==False:
            return 0
        else:
            return 2

    def get_input_data(self, row):
        # Preprocessing: {remove icon, special character, lower}
        text1 = row['question 1']
        text2 = row['question 2']
        label = self.labelencoder(row['similarity'])

        return text1, text2, label

In [ ]:
tokenizer_datasets = EquavilenceDataset(train_df, tokenizer, max_len=128)
tokenizer_validations = EquavilenceDataset(valid_df, tokenizer, max_len=128)

In [ ]:
tokenizer_datasets[1]

{'text1': 'người việt_nam bay vào vũ_trụ đầu_tiên tên là gì',
 'text2': 'ai là người vn đầu_tiên bay vào vũ_trụ',
 'input_ids': tensor([    0,    18, 56444,   542,   620,    33,  4196,   191,   221,     8,
           148,     2,     2,   277,     8,    18, 33756,   191,   620,    33,
          4196,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
    

In [ ]:
tokenizer_validations[3]

{'text1': 'album đầu_tay của adele tên gì',
 'text2': 'album đầu_tay của adele tên gì',
 'input_ids': tensor([    0,  2675,  7467,     7,  2811, 15156,   221,   148,     2,     2,
          2675,  7467,     7,  2811, 15156,   221,   148,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1, 

In [ ]:
from torch.utils.data import Dataset, DataLoader

train_loader = DataLoader(tokenizer_datasets, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(tokenizer_validations, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
class EquivalenceClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EquivalenceClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [ ]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    losses = []
    correct = 0

    for data in tqdm(train_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()

    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')

def eval(test_data = False):
    model.eval()
    losses = []
    correct = 0

    with torch.no_grad():
        data_loader = test_loader if test_data else valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())
    
    if test_data:
        print(f'Test Accuracy: {correct.double()/len(test_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(test_loader.dataset)
    else:
        print(f'Valid Accuracy: {correct.double()/len(valid_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(valid_loader.dataset)

In [ ]:
    fold = 0
    
    model = EquivalenceClassifier(n_classes=2).to(device)
    criterion = nn.CrossEntropyLoss()
    # Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
    # Batchsize: 16, 32
    optimizer = AdamW(model.parameters(), lr=2e-5)
    
    lr_scheduler = get_linear_schedule_with_warmup(
                optimizer, 
                num_warmup_steps=0, 
                num_training_steps=len(train_loader)*EPOCHS
            )
    best_acc = 0
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        print('-'*30)

        train(model, criterion, optimizer, train_loader)
        val_acc = eval()

        if val_acc > best_acc:
            torch.save(model.state_dict(), f'phobert_fold{fold+1}.pth')
            best_acc = val_acc

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/6
------------------------------


100%|██████████| 5448/5448 [32:35<00:00,  2.79it/s]

Train Accuracy: 0.9385820989708228 Loss: 0.1814297745914806


Valid Accuracy: 0.9783696529459242 Loss: 0.07635492277819844
Epoch 2/6
------------------------------


...

Train Accuracy: 0.9935403926248035 Loss: 0.02824391678059101


Valid Accuracy: 0.9858757062146893 Loss: 0.07002394887790526
Epoch 6/6
------------------------------


100%|██████████| 5448/5448 [32:34<00:00,  2.79it/s]

Train Accuracy: 0.9961907821517492 Loss: 0.01708189489478365


Valid Accuracy: 0.9870056497175141 Loss: 0.07039585694381986


In [242]:
from sklearn.metrics import classification_report, confusion_matrix

def test(data_loader):
    model = EquivalenceClassifier(n_classes=2)
    model.to(device)
    model.load_state_dict(torch.load(f'phobert_fold{fold+1}.pth'))
    model.eval()

    texts1 = []
    texts2 = []
    predicts = []
    predict_probs = []
    real_values = []

    for data in tqdm(data_loader):
        text1 = data['text1']
        text2 = data['text2']
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        total_outs = []

        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
        total_outs.append(outputs)
        
        total_outs = torch.stack(total_outs)
        _, pred = torch.max(total_outs.mean(0), dim=1)
        texts1.extend(text1)
        texts2.extend(text2)
        predicts.extend(pred)
        predict_probs.extend(total_outs.mean(0))
        real_values.extend(targets)
    
    predicts = torch.stack(predicts).cpu()
    predict_probs = torch.stack(predict_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    print(classification_report(real_values, predicts))
    return real_values, predicts

In [243]:
test_dataset = EquavilenceDataset(test_df, tokenizer, max_len=50)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)
real_values, predicts = test(test_loader)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2335/2335 [01:46<00:00, 21.88it/s]


              precision    recall  f1-score   support

           0       0.99      0.99      0.99     29215
           1       0.96      0.97      0.96      8139

    accuracy                           0.98     37354
   macro avg       0.97      0.98      0.98     37354
weighted avg       0.98      0.98      0.98     37354



In [229]:
class_names = ['False', 'True']

def infer(text1, text2, tokenizer, max_len=128):
    text1 = word_segment(text1)
    text2 = word_segment(text2)
    encoded_review = tokenizer.encode_plus(
        text1,
        text2,
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    print(f'Text1: {text1}')
    print(f'Text2: {text2}')
    print(f'Sentiment: {class_names[y_pred]}')

In [277]:
text1 = "tên thật của bác giáp là gì"
text2 = "tên thật của cụ võ nguyên giáp"

In [278]:
infer(text1, text2, tokenizer)

Text1: tên thật của bác giáp là gì
Text2: tên thật của cụ võ nguyên giáp
Sentiment: False
